# ReAct Agent Implementation with OpenAI

**Author**: Sergio Masa Avis  
**Date**: October 2025

Implementation of the ReAct (Reasoning + Acting) pattern using OpenAI's API. ReAct agent with single and multi-iteration execution modes.

---

## Table of Contents

1. [ReAct Pattern Overview](#react-pattern-overview)
2. [Architecture](#architecture)
3. [Implementation](#implementation)
4. [Examples](#examples)
5. [Limitations](#limitations)
6. [References](#references)

---

## 1. ReAct Pattern Overview

**ReAct** (Reasoning + Acting) is a prompting framework that enables LLMs to alternate between reasoning and action-taking to solve tasks.

### Core Mechanism

ReAct execution cycle:
1. **Thought**: Analyze current state and determine next action
2. **Action**: Execute tool or function call
3. **Observation**: Receive and process action result
4. **Repeat**: Continue until task completion

![ReAct Cycle](images/01_react_agent/react_cycle.png)

### Problem Statement

Direct LLM prompting limitations:
- No access to real-time data
- Cannot perform reliable calculations
- May hallucinate information

ReAct solutions:
- Tool access for data retrieval
- Computation delegation to deterministic functions
- Response grounding in actual observations

### Execution Modes

**Single Iteration**: One action per query

![Single Iteration](images/01_react_agent/single_iteration.png)

**Multi-Iteration**: Multiple actions until task completion

![Multi Iteration](images/01_react_agent/multi_iteration.png)

---

## 2. Architecture

System components:

![Architecture](images/01_react_agent/architecture_system.png)

### Data Flow

![Data Flow](images/01_react_agent/data_flow_sequence.png)

### Design Decisions

| Component | Implementation | Notes |
|-----------|----------------|-------|
| State Management | Stateful conversation history | Context preservation across interactions |
| Prompt Design | Explicit system instructions | Behavior definition through examples |
| Action Parsing | Regex pattern matching | String-based, limited flexibility |
| Execution Mode | Single-shot and looped | One action per query or multi-iteration |

---

## 3. Implementation

---

### 3.1 Environment Setup

Configuration:
- API key loaded from environment variables
- OpenAI client initialization
- Connection verification

In [1]:
import os
import re
from dotenv import load_dotenv
from openai import OpenAI

_ = load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "Missing OPENAI_API_KEY environment variable"

client = OpenAI()

# Verify connection
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say 'Hello, world!' and my name in English if you are ready."}],
)

print(f"Connection verified: {resp.choices[0].message.content}")

Connection verified: Hello, world! What's your name?


### 3.2 Agent Class

Properties:
- Conversation history management
- System prompt configuration
- OpenAI API integration
- Conversation History. 


#### Implementation Details

- `__init__`: Client and message history initialization
- `__call__`: `agent(message)` syntax support
- `execute`: API communication with temperature=0 for deterministic outputs

In [2]:
class Agent:
    """
    Stateful conversational agent with OpenAI integration.

    Maintains conversation history and manages system prompt configuration.
    """

    def __init__(self, system: str = ""):
        """
        Args:
            system: System prompt defining agent behavior
        """
        self.client = OpenAI()
        self.system = system
        self.messages = []

        if self.system:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message: str) -> str:
        """
        Process message and return response.

        Args:
            message: User input

        Returns:
            Agent response
        """
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self) -> str:
        """
        Execute API call with full conversation history.

        Returns:
            Model response content
        """
        completion = self.client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,  # Deterministic outputs (reliable parsing)
            messages=self.messages
        )
        return completion.choices[0].message.content

### 3.3 System Prompt

System prompt specification:
1. Execution loop (Thought → Action → PAUSE → Observation)
2. Available tools and their syntax
3. Example session with proper usage

![Prompt Anatomy](images/01_react_agent/prompt_anatomy.png)

#### Design Elements

**PAUSE keyword**: Parser signal for LLM observation awaiting. Prevents tool output hallucination.

**Specific examples**: Few-shot learning through detailed example sessions.

**Explicit syntax**: Clear tool invocation format, reduced parsing errors.

In [3]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

Use Thought to describe your thoughts about the question you have been asked.
IMPORTANT: In your Thought, always explicitly mention the SPECIFIC services, rates, or values you are working with.

Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate_project_cost:
e.g. calculate_project_cost: web: 40, data_science: 20
Runs a calculation for the total project cost based on hours and hourly rates of different services

get_service_rate:
e.g. get_service_rate: web
returns the hourly rate for the freelance service when given its name.

Example session:

Question: What is the total cost for 40 hours of web development and 20 hours of data science?
Thought: I need to find out the hourly rate for web development first, then the rate for data science, and finally calculate the total cost for 40 hours of web development and 20 hours of data science.
Action: get_service_rate: web
PAUSE

Observation: The hourly rate for web_development is $75

Thought: Now I know web development costs $75/hour. I need to get the rate for data science next.
Action: get_service_rate: data_science
PAUSE

Observation: The hourly rate for data_science is $95

Thought: Now I have both rates: web development is $75/hour and data science is $95/hour. I can calculate the total cost for 40 hours of web development and 20 hours of data science.
Action: calculate_project_cost: web: 40, data_science: 20
PAUSE

Observation: The total project cost is $4900

Answer: The total cost for 40 hours of web development and 20 hours of data science is $4900
""".strip()

print(f"System prompt configured ({len(prompt)} characters)")

System prompt configured (1751 characters)


### 3.4 Tools

Tool functions for agent invocation:
- String input from LLM
- String output for LLM consumption
- Graceful error handling

![Tool Design](images/01_react_agent/tool_design_principles.png)

#### Design Principles

| Principle | Implementation |
|-----------|---------------|
| String I/O | Text-based interfaces, LLM compatibility |
| Descriptive output | Full sentences, contextual information |
| Error handling | Informative failure messages |
| Logging | Execution debug visibility |

#### Implementation

![Tool Implementation](images/01_react_agent/tool_implementation.png)

In [4]:
# Rate database (USD/hour)
freelance_rates = {
    "web": 75,
    "data_science": 95,
    "mobile_app": 85,
    "ai_ml": 120,
    "backend_api": 80,
    "frontend": 70,
}

def get_service_rate(service: str) -> str:
    """
    Retrieve hourly rate for specified service.

    Args:
        service: Service name

    Returns:
        Rate information or error message
    """
    print(f"[get_service_rate] Query: {service}")
    service = service.lower().strip()

    if service in freelance_rates:
        return f"The hourly rate for {service} is ${freelance_rates[service]}"
    return f"Service '{service}' not found. Available: {', '.join(freelance_rates.keys())}"

def calculate_project_cost(services: str) -> str:
    """
    Calculate total cost for multiple services.

    Args:
        services: Format "service: hours, service: hours"

    Returns:
        Total cost or error message
    """
    print(f"[calculate_project_cost] Input: {services}")
    total = 0.0

    for item in services.split(', '):
        try:
            service, hours = item.split(': ')
            service = service.lower().strip()
            hours = int(hours)

            if service in freelance_rates:
                cost = freelance_rates[service] * hours
                total += cost
                print(f"  {hours}h {service} @ ${freelance_rates[service]}/h = ${cost}")
            else:
                return f"Service '{service}' not found"
        except ValueError:
            return f"Parse error: '{item}'. Expected format: 'service: hours'"

    print(f"  Total: ${total}")
    return f"The total project cost is ${total}"

known_actions = {
    "get_service_rate": get_service_rate,
    "calculate_project_cost": calculate_project_cost,
}

print(f"Tools registered: {list(known_actions.keys())}")

Tools registered: ['get_service_rate', 'calculate_project_cost']


### 3.5 Action Parsing

Action extraction and execution from LLM responses.

![Parsing Flow](images/01_react_agent/parsing_challenge.png)

#### Regex Pattern

```python
action_re = re.compile(r'^Action: (\w+): (.*)$')
```

| Component | Match |
|-----------|-------|
| `^Action:` | Line prefix |
| `(\w+)` | Tool name (captured) |
| `:` | Separator |
| `(.*)$` | Arguments (captured) |

Example:
```
Input:  "Action: get_service_rate: web_development"
Match:  tool="get_service_rate", args="web_development"
```

In [5]:
action_re = re.compile(r'^Action: (\w+): (.*)$')

### 3.6 Execution Modes

Two execution approaches for different use cases.

---

#### 3.6.1 Single Iteration

Execute one ReAct cycle: single tool invocation per query.

In [6]:
def query(question: str, verbose: bool = True):
    """
    Execute ReAct query with action parsing.

    Args:
        question: User query
        verbose: Enable detailed output

    Returns:
        Tool observation or None if no action found
    """
    bot = Agent(system=prompt)
    result = bot(question)

    if verbose:
        print("\n" + "="*70)
        print("AGENT RESPONSE")
        print("="*70)
        print(result)
        print("="*70 + "\n")

    # Parse actions from response
    actions = [
        action_re.match(line)
        for line in result.split('\n')
        if action_re.match(line)
    ]

    if actions:
        action, action_input = actions[0].groups()

        if action not in known_actions:
            raise ValueError(f"Unknown action: {action}")

        if verbose:
            print(f"EXECUTING: {action}({action_input})\n")

        observation = known_actions[action](action_input)

        if verbose:
            print(f"\nOBSERVATION: {observation}\n")

        return observation

    if verbose:
        print("No action found in response\n")
    return None

#### 3.6.2 Multi-Iteration Loop

Execute multiple ReAct cycles until task completion or loop limit reached.

![Multi-Iteration Control Flow](images/01_react_agent/multi_iteration_control.png)

In [ ]:
def query_with_loop(question: str, max_loops: int = 5):
    """
    Execute ReAct query with multiple iterations until completion.

    Args:
        question: User query
        max_loops: Maximum number of iterations

    Returns:
        Final agent response
    """
    i = 0
    bot = Agent(system=prompt)
    next_prompt = question

    while i < max_loops:
        i += 1
        result = bot(next_prompt)

        print("\n" + "=" * 70)
        print(f"AGENT RESPONSE - LOOP {i}/{max_loops}")
        print("=" * 70)
        print(result)
        print("=" * 70 + "\n")

        # Parse actions from response
        actions = [
            action_re.match(line) for line in result.split("\n") if action_re.match(line)
        ]

        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise ValueError(f"Unknown action: {action}")

            print(f"EXECUTING: {action}({action_input})\n")
            observation = known_actions[action](action_input)
            print(f"\nOBSERVATION: {observation}\n")

            next_prompt = f"Observation: {observation}\n"
        else:
            # Agent finished - return final response
            return result

    # Max loops reached - return last response
    print(f"⚠️ Max loops ({max_loops}) reached")
    return result

## 4. Examples

Agent behavior test cases.

---

### 4.1 Single Iteration Examples

---

#### 4.1.1 Simple Rate Lookup

Single tool invocation, data retrieval.

In [8]:
query("What is the hourly rate for AI/ML development?")


AGENT RESPONSE
Thought: I need to find out the hourly rate for AI/ML development specifically. I will perform an action to get the service rate for AI/ML development. 

Action: get_service_rate: ai_ml
PAUSE

EXECUTING: get_service_rate(ai_ml)

[get_service_rate] Query: ai_ml

OBSERVATION: The hourly rate for ai_ml is $120



'The hourly rate for ai_ml is $120'

#### 4.1.2 Unknown Service

Tool error handling, informative response.

In [9]:
query("What is the hourly rate for blockchain development?")


AGENT RESPONSE
Thought: I need to find out the hourly rate for blockchain development specifically. I will perform an action to retrieve this information. 
Action: get_service_rate: blockchain_development
PAUSE

EXECUTING: get_service_rate(blockchain_development)

[get_service_rate] Query: blockchain_development

OBSERVATION: Service 'blockchain_development' not found. Available: web, data_science, mobile_app, ai_ml, backend_api, frontend



"Service 'blockchain_development' not found. Available: web, data_science, mobile_app, ai_ml, backend_api, frontend"

#### 4.1.3 Multi-Step Query (Limited)

Single-iteration stops after first action despite requiring multiple steps.

In [10]:
query("What is the total cost for 40 hours of web development and 20 hours of data science?")


AGENT RESPONSE
Thought: I need to find out the hourly rate for web development first, then the rate for data science, and finally calculate the total cost for 40 hours of web development and 20 hours of data science. 

Action: get_service_rate: web
PAUSE

EXECUTING: get_service_rate(web)

[get_service_rate] Query: web

OBSERVATION: The hourly rate for web is $75



'The hourly rate for web is $75'

### 4.2 Multi-Iteration Examples

---

#### 4.2.1 Complete Multi-Step Calculation

Multi-iteration execution completing the full ReAct cycle.

In [11]:
query_with_loop("What is the total cost for 40 hours of web development and 20 hours of data science?")


AGENT RESPONSE - LOOP 1/5
Thought: I need to find the hourly rate for web development and data science to calculate the total cost for 40 hours of web development and 20 hours of data science. First, I will get the rate for web development. 

Action: get_service_rate: web
PAUSE

EXECUTING: get_service_rate(web)

[get_service_rate] Query: web

OBSERVATION: The hourly rate for web is $75


AGENT RESPONSE - LOOP 2/5
Thought: Now that I know the hourly rate for web development is $75, I need to find the hourly rate for data science to proceed with the total cost calculation. 

Action: get_service_rate: data_science
PAUSE

EXECUTING: get_service_rate(data_science)

[get_service_rate] Query: data_science

OBSERVATION: The hourly rate for data_science is $95


AGENT RESPONSE - LOOP 3/5
Thought: I now have both rates: web development is $75/hour and data science is $95/hour. I can calculate the total cost for 40 hours of web development and 20 hours of data science using these rates. 

Action

'Answer: The total cost for 40 hours of web development and 20 hours of data science is $4900.'

#### 4.2.2 Complex Project Estimation

Multiple service rates and calculations.

In [12]:
query_with_loop("I need a quote for a project with 30 hours of frontend work, 50 hours of backend API, and 25 hours of mobile app development. What's the total cost?")


AGENT RESPONSE - LOOP 1/5
Thought: I need to find the hourly rates for frontend development, backend API development, and mobile app development. Once I have those rates, I can calculate the total cost for 30 hours of frontend work, 50 hours of backend API work, and 25 hours of mobile app development. 

Action: get_service_rate: frontend
PAUSE

EXECUTING: get_service_rate(frontend)

[get_service_rate] Query: frontend

OBSERVATION: The hourly rate for frontend is $70


AGENT RESPONSE - LOOP 2/5
Thought: I now know that frontend development costs $70/hour. Next, I need to get the rate for backend API development. 

Action: get_service_rate: backend_api
PAUSE

EXECUTING: get_service_rate(backend_api)

[get_service_rate] Query: backend_api

OBSERVATION: The hourly rate for backend_api is $80


AGENT RESPONSE - LOOP 3/5
Thought: Now I have the rate for backend API development, which is $80/hour. Next, I need to find the rate for mobile app development. 

Action: get_service_rate: mobile_ap

'Answer: The total cost for 30 hours of frontend work, 50 hours of backend API work, and 25 hours of mobile app development is $8225.'

## 5. Limitations

Implementation constraints and considerations.

---

### String Parsing Fragility

Regex-based parsing fails on format variations:

| Format | Status |
|--------|--------|
| `"Action: tool: input"` | Valid |
| `"Action: tool : input"` | Invalid (space) |
| `"Action:tool: input"` | Invalid (no space) |
| `"action: tool: input"` | Invalid (case) |

**Alternative**: OpenAI function calling for structured outputs.

---

### Loop Control

Multi-iteration mode relies on hard loop limits rather than intelligent termination detection.

**Limitation**: May stop prematurely or continue unnecessarily.

**Alternative**: Answer pattern detection for dynamic termination.

---

### Additional Constraints

| Issue | Impact | Alternative |
|-------|--------|-------------|
| No retry logic | Single failure = task failure | Error recovery implementation |
| No validation | Tools can crash silently | Type checking + try-catch |
| Hardcoded tools | Requires code changes | Dynamic tool registry |
| No persistence | Session-bound | State storage implementation |
| No cost tracking | Unknown spending | Token usage monitoring |

---

## 6. References

**Papers**:
- [ReAct: Synergizing Reasoning and Acting](https://arxiv.org/abs/2210.03629)
- [Toolformer: Language Models Can Teach Themselves to Use Tools](https://arxiv.org/abs/2302.04761)

**Documentation**:
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [LangGraph](https://langchain-ai.github.io/langgraph/)

---

*Last updated: October 2025*